
# 🧠 Pipeline 2 – Análise de Sentimentos em Comentários do YouTube

Este notebook implementa a segunda fase do projeto **Análise de Sentimentos em Comentários de Vídeos do YouTube**.

A partir dos comentários coletados na *Pipeline 1 (coleta.ipynb)*, aplicamos técnicas de **Processamento de Linguagem Natural (NLP)** para classificar os sentimentos expressos pelos usuários em **positivos, negativos ou neutros**.

📘 **Etapa CRISP-DM:** Modeling  
📘 **Fase anterior:** Coleta de Dados  
📘 **Próxima fase:** Visualização e Integração Flask
    

In [5]:

# 🔧 Importação das bibliotecas
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import pipeline
from langdetect import detect
import re
    

In [7]:

# ⚙️ Carregar os dados coletados
df = pd.read_csv("comentarios_top5.csv")

print("✅ Dataset carregado com sucesso!")
print(f"Total de comentários: {len(df)}")
df.head()
    

FileNotFoundError: [Errno 2] No such file or directory: 'comentarios_top5.csv'

In [ ]:

# 🧹 Pré-processamento dos textos
def limpar_texto(texto):
    texto = re.sub(r"http\S+", "", texto)
    texto = re.sub(r"@\w+", "", texto)
    texto = re.sub(r"[^A-Za-zÀ-ÿ\s]", "", texto)
    texto = re.sub(r"\s+", " ", texto)
    return texto.strip()

df["comentario_limpo"] = df["comentario"].astype(str).apply(limpar_texto)
print("🧹 Pré-processamento concluído!")
df.head()
    

In [ ]:

# 🌍 Detecção e filtragem de idioma
def detectar_idioma(texto):
    try:
        return detect(texto)
    except:
        return "erro"

df["idioma"] = df["comentario_limpo"].apply(detectar_idioma)

df_pt = df[df["idioma"] == "pt"].copy()

print(f"Comentários totais: {len(df)}")
print(f"Comentários em português: {len(df_pt)}")
df_pt.head()
    

In [ ]:

# 🧠 Carregar o modelo de análise de sentimentos
analisador = pipeline("sentiment-analysis",
                      model="pierreguillou/bert-base-cased-sentiment-br")
    

In [ ]:

# 🔍 Aplicar análise de sentimentos
resultados = analisador(list(df_pt["comentario_limpo"]))

resultados_df = pd.DataFrame(resultados)
df_pt["sentimento"] = resultados_df["label"]
df_pt["score"] = resultados_df["score"]

print("✅ Análise de sentimentos concluída!")
df_pt.head()
    

In [ ]:

# 📊 Estatísticas descritivas
dist = df_pt["sentimento"].value_counts(normalize=True) * 100

print("Distribuição percentual dos sentimentos:")
print(dist.round(2))

sns.set(style="whitegrid")
plt.figure(figsize=(6,4))
sns.countplot(x="sentimento", data=df_pt, palette="viridis")
plt.title("Distribuição dos Sentimentos nos Comentários")
plt.xlabel("Sentimento")
plt.ylabel("Quantidade")
plt.show()
    

In [ ]:

# 📈 Comparação entre vídeos
plt.figure(figsize=(10,6))
sns.countplot(x="video_titulo", hue="sentimento", data=df_pt, palette="viridis")
plt.title("Comparativo de Sentimentos por Vídeo")
plt.xticks(rotation=45, ha="right")
plt.xlabel("Vídeos")
plt.ylabel("Número de Comentários")
plt.legend(title="Sentimento")
plt.tight_layout()
plt.show()
    

In [ ]:

# 💾 Exportar resultados
df_pt.to_csv("comentarios_analisados.csv", index=False)
print("📂 Resultados exportados: comentarios_analisados.csv")
    


# 🔎 Conclusão

A **Fase 2 – Análise de Sentimentos** foi implementada com sucesso.

O modelo de linguagem pré-treinado da HuggingFace foi aplicado aos comentários coletados, classificando-os como **positivos, negativos ou neutros**.  
O conjunto de dados agora contém as colunas:

- `video_id`
- `video_titulo`
- `comentario`
- `comentario_limpo`
- `idioma`
- `sentimento`
- `score`

Os resultados obtidos servirão como base para a **Fase 3 (Visualização e Integração Flask)**, onde será desenvolvido o dashboard interativo e a interface web de análise.
    